In [4]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import random
import time

In [1]:
class board:
    
    files = ['a','b','c','d','e','f','g','h']
    ranks = [1,2,3,4,5,6,7,8]
    moveDict = {'upx':('up','x'),
               'downx':('down','x'),
               'leftx':('left','x'),
               'rightx':('right','x'),
               'upxleftx':(('up','x'),('left','x'),2),
               'upxrightx':(('up','x'),('right','x'),2),
               'downxleftx':(('down','x'),('left','x'),2),
               'downxrightx':(('down','x'),('right','x'),2),
               'up1':('up',1),
               'down1':('down',1),
               'left1':('left',1),
               'right1':('right',1),
               'up1left1':(('up',1),('left',1),2),
               'up1right1':(('up',1),('right',1),2),
               'down1left1':(('down',1),('left',1),2),
               'down1right1':(('down',1),('right',1),2),
               'up2left':(('up',2),('left',1),2),
               'up2right':(('up',2),('right',1),2),
               'left2up':(('left',2),('up',1),2),
               'left2down':(('left',2),('down',1),2),
               'right2up':(('right',2),('up',1),2),
               'right2down':(('right',2),('down',1),2),
               'down2left':(('down',2),('left',1),2),
               'down2right':(('down',2),('right',1),2),
               'up2':('up',2),
                'down2':('down',2),
               'WpTakeL':(('up',1),('left',1),2),
               'WpTakeR':(('up',1),('right',1),2),
               'BpTakeL':(('down',1),('left',1),2),
               'BpTakeR':(('down',1),('right',1),2),
               'castleR': ('right',2),
               'castleL': ('left',2)}
    
    def __init__(self, read=False, imaginary=False, copyFrom=False):
        #creates a board as a DataFrame or clones an existing board
        if read:
            self.body = pd.read_csv(read, index_col=0)
        if imaginary:
            self.body = copyFrom.copy()
            
    def recon(self, rsquare):
        #look at a square and return the piece ID or False
        if not rsquare:
            return False
        if rsquare == '0':
            return False
        #print('recon rsquare: ' + str(rsquare))
        file = rsquare[0]
        rank = rsquare[1]
        square = self.body.at[rank,file]
        #print('recon square: ' + str(square))
        return square
            
    def walk(self, square, movement):
        if not square:
            return False
        file = square[0]
        rank = square[1]
        direction = movement[0]
        steps = movement[1]
        if steps == 'x':
            steps = 1
        #print(steps)
        fileCom = {"up":1, "down":-1, "left":-1, "right":1}
        global newPos
        if direction in ["left", "right"]:
            findex = board.files.index(file)
            nindex = findex + (steps * fileCom[direction])
            if nindex not in range(0,8):
                return False
            newFile = board.files[nindex]
            newPos = (newFile, rank)
        if direction in ["up", "down"]:
            newRank = (rank + (steps * fileCom[direction]))
            if newRank not in range(1,9):
                return False
            newPos = (file, newRank)
        return newPos
    
    def movePiece(self, square, newSquare):
        self.body.at[newSquare[1], newSquare[0]] = b1.body.at[square[1], square[0]]
        self.body.at[square[1], square[0]] = '0'
    
    def assessSquare(self, square, newSquare, origin):
        #print('ASSESSSQUARE')
        check = self.recon(newSquare)
        if not check:
            #out of bounds
            #print('oob')
            return 'oob'
        elif check is '0':
            #then the square is empty
            #print('empty')
            return 'empty'
        else:
            #the square is either friend or foe
            #print(recon(newSquare))
            #print(recon(square))
            if self.recon(newSquare)[0] is not self.recon(origin)[0]:
                #print('foe')
                return 'foe'
            else:
                #print('friend')
                return 'friend'
    
    def xCheck(self, x, move, square, imaginary=False):
        #print('XCHECK')
        origin = square
        #look through each move str name, make moves and report if possible
        if 'x' in move:

            #print(move + ': x')
            #need to test further moves in this direction until blocked
            blocked = False
            xSquares = []
            while not blocked:
                #print('not blocked')
                if len(x) == 3:
                    #move is a two-step
                    move1 = x[0]
                    #print(move1)
                    move2 = x[1]
                    #print(move2)
                    newSquare = self.walk(square, move1)
                    #print(newSquare)
                    newSquare = self.walk(newSquare, move2)
                    #print(newSquare)
                else:
                    #move is a one-step
                    newSquare = self.walk(square, x)
                    #print(newSquare)

                result = self.assessSquare(square, newSquare, origin)
                #print(result)
                if result == 'oob':
                    blocked = True
                    newSquare = False
                elif result == 'empty':
                    blocked = False
                    xSquares.append(newSquare)
                    square = newSquare
                elif result == 'friend':
                    blocked = True
                    newSquare = False
                else:
                    blocked = True
                    xSquares.append(newSquare)
                    square = newSquare
            newSquare = xSquares
            if len(newSquare) == 0:
                newSquare = False

        if 'x' not in move:
            #print(move + ': no x')
            #move only specified number of moves

            #only for castle
            if 'castle' in move:
                if 'L' in move:
                    team = 'l'
                if'R' in move:
                    team = 'r'

            #non-castle
            if len(x) == 3:
                #print(x)
                #move is a two-step
                #this is where the horses move
                move1 = x[0]
                #print(move1)
                move2 = x[1]
                #print(move2)
                newSquare = self.walk(square, move1)
                newSquare = self.walk(newSquare, move2)
            else:
                #move is a one-step
                newSquare = self.walk(square, x)
            result = self.assessSquare(square, newSquare, origin)
            if result == 'oob':
                newSquare = False
            if result == 'friend':
                newSquare = False
            if 'P' in self.recon(square):
                if newSquare:
                    if not imaginary:
                        if newSquare[0] is not square[0]:
                            if self.recon(newSquare) is '0':
                                newSquare = False
                        if newSquare[0] is square[0]:
                            if self.recon(newSquare) is not '0':
                                newSquare = False
                    if imaginary:
                        if newSquare[0] is square[0]:
                            if self.recon(newSquare) is not '0':
                                newSquare = False
                        if newSquare[0] is not square[0]:
                            if self.recon(newSquare) is '0':
                                newSquare = newSquare
        return newSquare
    
    def moveTester(self, square, castle=False, imaginary=False):
        #returns legal moves
        #print('MOVETESTER')
        legalmoves = {}
        if self.recon(square) == '0':
            return False
        moves = self.defaultMoves(square, castle=castle)
        #print(moves)
        for move in moves:
            x = board.moveDict[move]
            #print(move)
            #print(x)
            newSquare = self.xCheck(x, move, square, imaginary)
            legalmoves[move] = newSquare

        #returns a dict of moves and their resulting squares (or False)
        return legalmoves
    
    def moveCounter(self, square, castle=False, imaginary=False):
        #print('MOVECOUNTER')
        #returns count of legal moves
        count = 0
        movesReturned = self.moveTester(square, castle=castle, imaginary=imaginary)
        #print(movesReturned)
        if movesReturned:
            for move in movesReturned.items():
                #print(move[1])
                if move[1]:
                    #print(move)
                    count += 1
                    if len(move) > 2:
                        count += (len(move) - 1)
        return count
    
    def myTeam(self, color, castle=False, imaginary=False):
        #global board
        #print('MYTEAM')
        #team is w or b (str)
        myPieces = {}
        for rank in board.ranks:
            for file in board.files:
                piece = self.body.at[rank,file]
                #print(piece)
                if color in piece:
                    movenum = self.moveCounter((file,rank), castle, imaginary)
                    myPieces[piece + str(file) + str(rank)] = ((file, rank), movenum)
        return myPieces
    
    def myMoves(self, teamdict):
        #print('MYMOVES')
        moves = {}
        for piece in teamdict:
            x = teamdict[piece]
            if x[1] is not 0:
                moves[piece] = x
        return moves
    
    def pickRandomMove(self, team, castle=False, specific=False):
        canmoves = []
        x = self.myMoves(self.myTeam(team))
        if len(x) == 0:
            return 'done', 'done'
        #print(x)
        while len(canmoves) == 0:
            if specific:
                pick = specific
            if not specific:
                pick = random.choice(list(x))
            #print('pick: ' + str(pick))
            square = x[pick][0]
            moves = self.moveTester(square, castle)
            #print(moves)
            for y in list(moves):
                if moves[y]:
                    canmoves.append(y)
        #print(canmoves)    
        movepick = random.choice(canmoves)
        return pick, movepick
    
    def dangerDetector(self, square, team, imaginary=False):
        attacks = []
        enemy = False
        if team == 'w':
            enemy = 'b'
        if team == 'b':
            enemy = 'w'
        if not enemy:
            return False
        for rank in board.ranks:
            for file in board.files:
                if enemy in self.recon((file,rank)):
                    #print('enemy: ' + str(recon((file,rank))) + str(file) + str(rank))
                    moves = self.moveTester((file,rank), imaginary=imaginary)
                    #print(moves)
                    for move in moves.values():
                        if move:
                            #print('square: ' + str(square))
                            #print('move: ' + str(move))
                            if str(square) in str(move):
                                loc = (file,rank)
                                attacks.append(str(self.recon(loc)) + str(file) + str(rank))
        if len(attacks) == 0:
            return False
        return attacks
    
    def canCastle(self, team, side):
        #check for danger or teammate along king path
        dangers = 0
        if team == 'w':
            if side == 'l':
                #king is at ('e',1), rook is at ('a',1)
                #king goes to ('c',1) and left rook goes to ('d',1)
                #e1 thru c1 must not be in danger
                path = [('e',1),('d',1),('c',1)]

            if side == 'r':
                #king is at ('e',1), rook is at ('h',1)
                #king goes to ('g',1) and right rook goes to ('f',1)
                #e1 thru g1 must not be in danger
                path = [('e',1),('f',1),('g',1)]

        if team == 'b':
            if side == 'l':
                #king is at ('e',8), rook is at ('a',8)
                #king goes to ('c',8) and left rook goes to ('d',8)
                #e8 thru c8 must not be in danger
                path = [('e',8),('d',8),('c',8)]

            if side == 'r':
                #king is at ('e',8), rook is at ('h',8)
                #king goes to ('g',8) and rook goes to ('f',8)
                #e8 thru g8 must not be in danger
                path = [('e',8),('f',8),('g',8)]
        for spot in path:
            piece = self.recon(spot)
            if 'K' not in piece:
                if '0' not in piece:
                    dangers += 1
                if '0' in piece:
                    x = self.dangerDetector((spot), team)
                    if x:
                        dangers += len(x)
            #print(dangers)
        if dangers == 0:
            return True
        else:
            return False
        
    def castle(team, side):
        #global board
        if team == 'w':
            if side == 'l':
                #king is at ('e',1), rook is at ('a',1)
                #king goes to ('c',1) and left rook goes to ('d',1)
                kingfrom = ('e',1)
                kingto = ('c',1)
                rookfrom = ('a',1)
                rookto = ('d',1)
                ('white queen side castle')

            if side == 'r':
                #king is at ('e',1), rook is at ('h',1)
                #king goes to ('g',1) and right rook goes to ('f',1)
                kingfrom = ('e',1)
                kingto = ('g',1)
                rookfrom = ('h',1)
                rookto = ('f',1)
                ('white king side castle')

        if team == 'b':
            if side == 'l':
                #king is at ('e',8), rook is at ('a',8)
                #king goes to ('c',8) and left rook goes to ('d',8)
                kingfrom = ('e',8)
                kingto = ('c',8)
                rookfrom = ('a',8)
                rookto = ('d',8)
                ('black queen side castle')

            if side == 'r':
                #king is at ('e',8), rook is at ('h',8)
                #king goes to ('g',8) and rook goes to ('f',8)
                kingfrom = ('e',8)
                kingto = ('g',8)
                rookfrom = ('h',8)
                rookto = ('f',8)
                print('black king side castle')
    
        ### move king ###
        self.body.at[kingto[1],kingto[0]] = self.body.at[kingfrom[1],kingfrom[0]]
        self.body.at[kingfrom[1],kingfrom[0]] = '0'
        #################
        ### move rook ###
        self.body.at[rookto[1],rookto[0]] = self.body.at[rookfrom[1],rookfrom[0]]
        self.body.at[rookfrom[1],rookfrom[0]] = '0'
        #################
        #return self.body

    def defaultMoves(self, square, castle=False):
        moves = []
        #print('DEFAULTMOVES')
        #returns default moveset for a piece
        file = square[0]
        rank = square[1]
        piece = self.recon(square)
        if piece == '0':
            return False
        color = piece[0]
        #print(color)
        pieceType = piece[1]
        #print(pieceType)
        if pieceType == 'P':
            #check pawn team and loc
            if color == 'w':
                moves = ['up1','WpTakeL','WpTakeR']
                if rank == 2:
                    #pawn can move up 1 or 2
                    moves = ['up1','up2','WpTakeL','WpTakeR']
            if color == 'b':
                moves = ['down1','BpTakeL','BpTakeR']
                if rank == 7:
                    #pawn can move down 1 or 2
                    moves = ['down1','down2','BpTakeL','BpTakeR']
        if pieceType == 'R':
            moves = ['upx','downx','leftx','rightx']
        if pieceType == 'N':
            moves = ['up2left','up2right','left2up','left2down','right2up','right2down','down2left','down2right']
        if pieceType == 'B':
            moves = ['upxleftx','upxrightx','downxleftx','downxrightx']
        if pieceType == 'Q':
            moves = ['upx','downx','leftx','rightx','upxleftx','upxrightx','downxleftx','downxrightx']
        if pieceType == 'K':
            if not castle:
                moves = ['up1','down1','left1','right1','up1left1','up1right1','down1left1','down1right1']
            if castle:
                left = self.canCastle(color, 'l')
                right = self.canCastle(color,'r')
                moves = ['up1','down1','left1','right1','up1left1','up1right1','down1left1','down1right1']
                if left and right:
                    moves.extend(['castleL', 'castleR'])
                elif left:
                    moves.extend(['castleL'])
                elif right:
                    moves.extend(['castleR'])
        return moves
    
    def promote(self, team, square):
        #global board
        options = ['Q', 'R', 'B', 'N']
        promotion = random.choice(options)
        newPiece = str(team) + promotion
        self.body.at[square[1],square[0]] = newPiece
        print(str(team) + 'P' + str(square[0]) + str(square[1]) + ' promoted to ' + promotion)
        #print(newPiece)

In [ ]:
class future:
    
    def __init__(self, steps, copyFrom, anyMove=True):
        clone = board(copyFrom)

In [6]:
b1 = board("chess board avoidchecktest.csv")

In [13]:
b1.body

,a,b,c,d,e,f,g,h
8,bR,bN,bB,0,bK,bB,bN,bR
7,bP,bP,bP,bP,bP,bP,bP,bP
6,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0
4,0,0,0,0,0,bQ,0,0
3,0,0,0,0,wK,0,0,0
2,wP,wP,wP,wP,wP,wP,wP,wP
1,wR,wN,wB,wQ,0,wB,wN,wR


In [8]:
b2 = board(imaginary=True, copyFrom=b1.body)

In [12]:
b2.body

,a,b,c,d,e,f,g,h
8,bR,bN,bB,0,bK,bB,bN,bR
7,bP,bP,bP,bP,bP,bP,bP,bP
6,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0
4,0,0,0,0,0,bQ,0,0
3,0,0,0,0,wK,0,0,0
2,wP,wP,wP,wP,wP,wP,wP,wP
1,wR,wN,wB,wQ,0,wB,wN,wR


In [97]:
b1.walk(('e',2), ('up',2))

('e', 4)

In [98]:
b1.recon(('e',2))

'wP'

In [99]:
b1.promote('w', ('h',2))

wPh2 promoted to R


In [10]:
def play(steps=False, resetBoard=False):
    global turn
    #global b1
    global wKmoved
    global bKmoved
    global wRmoved
    global bRmoved
    wKmoved = False
    bKmoved = False
    wRmoved = {'l':False,'r':False}
    bRmoved = {'l':False,'r':False}
    count = 0
    over = False
    if resetBoard:
        b1 = board("chess board avoidchecktest.csv")
        turn = 'w'
    if steps:
        step = 0
    while True: 
        ### look at team ###
        team = b1.myTeam(turn)

In [11]:
play(10000, resetBoard=True)

KeyboardInterrupt: 

In [119]:
b1.body

,a,b,c,d,e,f,g,h
8,bR,bN,bB,0,bK,bB,bN,bR
7,bP,bP,bP,bP,bP,bP,bP,bP
6,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0
4,0,0,0,0,0,bQ,0,0
3,0,0,0,0,wK,0,0,0
2,wP,wP,wP,wP,wP,wP,wP,wP
1,wR,wN,wB,wQ,0,wB,wN,wR
